# Extracting Data Through API and storing in a variable "Data"

In [35]:
import json
import pandas as pd
import requests
from copy import deepcopy
from datetime import datetime
import time

False

In [23]:
BASE_URL = "https://acsyshf8au-dsn.algolia.net/1/indexes/*/queries"

In [24]:
HEADERS = {
    "content-type" : "application/json",
    "accept" : "application/json",
    "accept-language" : "en-US",
    "origin" : "https://www.freshco.com",
    "referer" : "https://www.freshco.com/",
    "user-agent" : "Mozilla/5.0",
    "x-algolia-api-key" : "425f09dfeda73e59bc3630b634257242",
    "x-algolia-application-id" : "ACSYSHF8AU"
}

In [25]:
BASE_PAYLOAD = {
    "requests":[
        {
            "indexName":"dxp_product_en",
            "analyticsTags":["H","website"],
            "clickAnalytics":True,
            "facetFilters":[
                ["hierarchicalCategories.lvl0:Fresh Fruits & Vegetables"]],
            "facets":[
                "brand",
                "hierarchicalCategories.lvl0",
                "hierarchicalCategories.lvl1",
                "price",
                "promotions.endDate",
                "promotions.promotionComplianceFlag",
                "promotions.promotionReward.rewardType",
                "promotions.source",
                "promotions.startDate",
                "recipeTags"
                ],
            f"filters":"storeId: 9636 AND allergens.alcohol:false AND isVisible:true AND isMassOffers:false AND hierarchicalCategories.lvl0:\"Fresh Fruits & Vegetables\"",
            "highlightPostTag":"__/ais-highlight__",
            "highlightPreTag":"__ais-highlight__",
            "hitsPerPage":15,
            "maxValuesPerFacet":1000,
            "page":0,
            "userToken":"anonymous-f3e3b0bd-84d5-4488-a315-fa87f571babb"
            },
        ]
    }

In [31]:


def extract_hits(base_url, headers, base_payload, source_name="freshco_algolia"):
    raw_hits = []
    page = 0
    extracted_at = datetime.utcnow().isoformat()

    while True:
        base_payload["requests"][0]["page"] = page

        response = requests.post(base_url, headers=headers, json=base_payload, timeout=30)
        response.raise_for_status()
        data = response.json()

        hits = data["results"][0].get("hits", [])
        if not hits:
            break

        # store each hit as JSON string to avoid type issues
        for hit in hits:
            raw_hits.append({
                "payload": json.dumps(hit),
                "extracted_at": extracted_at,
                "source": source_name
            })

        if page >= data["results"][0].get("nbPages", 0) - 1:
            break
        page += 1
    
    return raw_hits


In [33]:
Data = extract_hits(BASE_URL,HEADERS,BASE_PAYLOAD)

# Storing Data Into Azure Blob Storage As .parquet

In [73]:
! pip install azure-storage-blob
! pip install pyarrow

In [65]:
import io
import os
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv


In [66]:
os.getcwd()

'/Users/pankajkumar/Desktop/InventoryAnalysis/Python'

### Converting our data into df with three columns Payload, extracted_at and source

In [41]:
df = pd.DataFrame(Data)

### Accessing all the credential from a secure .env files

In [77]:
load_dotenv()
ACCOUNT_NAME = os.environ.get("ACCOUNT_NAME")
CONTAINER_NAME = os.environ.get("CONTAINER_NAME")
BLOB_NAME = os.environ.get("BLOB_NAME")
SAS_TOKEN = os.environ.get("SAS_TOKEN")

### Connecting to storage azure storage account

In [71]:
account_url = f"https://{ACCOUNT_NAME}.blob.core.windows.net" # Azure Account Name

blob_service = BlobServiceClient(
    account_url=account_url,
    credential=SAS_TOKEN
)


### Converting df to .parquet before storing into the actual storage

In [72]:

parquet_buffer = io.BytesIO()
df.to_parquet(parquet_buffer, engine="pyarrow", index=False)
parquet_buffer.seek(0)

0

### Uploading data to the storage as .parquet file
### Overwriting if it already exists

In [74]:
blob_client = blob_service.get_blob_client(
    container=CONTAINER_NAME,
    blob=BLOB_NAME
)

blob_client.upload_blob(parquet_buffer, overwrite=True)

print("Upload successful")

Upload successful
